In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os



In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
data = pd.read_csv('/kaggle/input/did-it-rain-in-seattle-19482017/seattleWeather_1948-2017.csv') 
data.head()

# Data Cleaning

### We need to drop particular columns, that we don't need. 
- Date, but we shall extract day, month and year instead
- PRCP (otherwise it would be a cheating)


In [3]:
#1. Drop the PRCP
data=data.drop(["PRCP"],axis=1)

# 2 Converting object into datetime to extract day, month and year
from datetime import datetime
data["DATE"]=pd.to_datetime(data["DATE"], format= "%Y-%m-%d")

# Extract day, month and year
data["DAY"]=data["DATE"].dt.day
data["MONTH"]=data["DATE"].dt.month
data["YEAR"]=data["DATE"].dt.year
data=data.drop(["DATE"], axis=1)

#Rearrange columns
data=data[["DAY", "MONTH", "YEAR", "TMAX", "TMIN", "RAIN"]]
data.head()

In [4]:
data.tail()

In [5]:
x=data.iloc[:,:-1].values
y=data.iloc[:,-1].values

## Encoding 

In [6]:
#Label Encoding for RAIN column
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(y)
y

## Splitting the dataset for Training and Test set

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

##  Feature Scaling

In [8]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
x_train=ss.fit_transform(x_train)
x_test=ss.transform(x_test)

## Building the ANN

In [24]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu', input_dim = 5))

# Adding the second hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the third hidden layer
classifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(x_train, y_train, batch_size = 10, epochs = 100)

## Prediction: 

- Day : 16
- Month: 11
- Year: 1992
- TMAX:54
- TMIN: 30


In [20]:
ann.predict(ss.transform([[16,11,1992,54,30]]))>0.5


## Predicting the test set results

In [21]:
y_pred=ann.predict(x_test)
y_pred=(y_pred>0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

## Present Confusion Matrix and Accuracy Score

In [22]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm=confusion_matrix(y_test, y_pred)
cm

In [23]:
accuracy_score(y_test, y_pred)

## Graphs

In [ ]:
from pylab import rcParams

rcParams['figure.figsize'] = 8, 4
plt.barh(ML_name, ML_accuracy, color = 'purple')
plt.xlabel('Accuracy Score', fontsize = '14')
plt.ylabel('Machine Learning Algorithms', fontsize = '14')
plt.xlim([0.7, 0.84])
plt.show()
#code is taken from here: https://www.kaggle.com/ehsaner/predicting-seattle-rain-part-2